In [ ]:
!pip install --upgrade --no-deps --force-reinstall kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d hj5992/restaurantreviews

     |████████████████████████████████| 61kB 4.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=8f1c1b10195496a7ef0c269d5b2cb2b7e4d4008f7074b2db9becf139072ba597
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Saving kaggle.json to kaggle.json
  0% 0.00/24.1k [00:00<?, ?B/s]
100% 24.1k/24.1k [00:00<00:00, 22.5MB/s]


In [ ]:
file_name='restaurantreviews.zip'
from zipfile import ZipFile
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Extraction Done')

Extraction Done


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from IPython.display import clear_output
clear_output(wait=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
df=pd.read_csv('./Restaurant_Reviews.tsv', encoding='latin-1',sep='\t')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]
sentences=df.Review
stemmer=PorterStemmer()
for i in range(len(sentences)):
  sent=re.sub('[^A-Za-z]',' ',sentences[i])
  sent=sent.lower()
  words=nltk.word_tokenize(sent)
  words=[stemmer.stem(word) for word in words if word not in stopwords.words('english')]
  sent=' '.join(words)
  corpus.append(sent)

In [ ]:
joblib.dump(corpus,"corpus.pkl")


['corpus.pkl']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(ngram_range=(1,2),max_features=5000)
corpus=cv.fit_transform(corpus).toarray()

In [ ]:
(corpus)

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch',
 'servic prompt',
 'would go back',
 'cashier care ever say still end wayyy overpr',
 'tri cape cod ravoli chicken cranberri mmmm',
 'disgust pretti sure human hair',
 'shock sign indic cash',
 'highli recommend',
 'waitress littl slow servic',
 'place worth time let alon vega',
 'like',
 'burritto blah',
 'food amaz',
 'servic also cute',
 'could care less interior beauti',
 'perform',
 'right red velvet cake ohhh stuff good',
 'never brought salad ask',
 'hole wall great mexican street taco friendli staff',
 'took hour get food tabl restaur food luke warm sever run around like total overwhelm',
 'worst salmon sashimi',
 'also combo like burger fri beer decent deal',
 'like final blow',
 'found place acc

['corpus.pkl']

In [ ]:
from sklearn.pipeline import Pipeline
estimators = []
estimators.append(('clean', clean_text()))
estimators.append(('CountVectorizer', CountVectorizer(ngram_range=(1,2),max_features=5000)))
model = Pipeline(estimators)
model.fit()

TypeError: ignored

In [ ]:
y=df.Liked.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.2,random_state=42)

from sklearn.linear_model import LogisticRegression
logistic_regression=LogisticRegression()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.75
              precision    recall  f1-score   support

           0       0.69      0.85      0.77        96
           1       0.83      0.65      0.73       104

    accuracy                           0.75       200
   macro avg       0.76      0.75      0.75       200
weighted avg       0.76      0.75      0.75       200

0.7311827956989247
[[82 14]
 [36 68]]


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
gaussion_nb=GaussianNB()
parameters={'var_smoothing':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
cv=GridSearchCV(gaussion_nb,parameters,scoring='accuracy',cv=4)
cv.fit(corpus,y)
print(cv.best_params_)
print(cv.best_score_)

y_pred=cv.predict(X_test)
print("accuracy ==>",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

{'var_smoothing': 0.01}
0.7229999999999999
accuracy ==> 0.98
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        96
           1       0.96      1.00      0.98       104

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200

0.9811320754716981
[[ 92   4]
 [  0 104]]


In [ ]:
from sklearn.externals import joblib
joblib.dump(cv,"restaurent_review_model.pkl")

['restaurent_review_model.pkl']